In [12]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import os

### Dataset
The data I used in this notebook can be found here, on Kaggle: https://www.kaggle.com/shayanfazeli/heartbeat

In [2]:
path = 'dataset/mitbih_train.csv'
df = pd.read_csv(path, header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87554 entries, 0 to 87553
Columns: 188 entries, 0 to 187
dtypes: float64(188)
memory usage: 125.6 MB


In [4]:
# Let's see the classes in which we have to classify our heartbeats
df[187].value_counts()

0.0    72471
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: 187, dtype: int64

In [ ]:
'''This piece of code takes the numerical values from the dataframe and generates for each row an image, which we'll use later
as input of the CNN.'''
'''I changed the name of the folder (0,1,2,3,4) manually, to split the data in train, validation and test sets. So I didn t use 
the csv file for testing, but I created my own version (images of course) from the train data, because the were enough.'''

for count, i in enumerate(df.values[81123:]):
    fig = plt.figure(frameon=False)
    plt.plot(i) 
    plt.xticks([]), plt.yticks([])
    for spine in plt.gca().spines.values():
        spine.set_visible(False)

    filename = 'img_dataset/4/' + '4' + '-' + str(count) + '.png'
    fig.savefig(filename)
    im_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    im_gray = cv2.resize(im_gray, (512, 512), interpolation = cv2.INTER_LANCZOS4)
    cv2.imwrite(filename, im_gray)

C:\Miniconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [1]:
import tensorflow as tf
from tensorflow import gfile
from tensorflow import compat
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

Using TensorFlow backend.


In [4]:
def create_model():
    K.set_image_dim_ordering('tf')

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(512, 512, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    return model

In [5]:
model = create_model()

batch_size = 64

# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        horizontal_flip=True)

# This is the augmentation configuration we will use for testing:
test_datagen = ImageDataGenerator(rescale=1./255)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

train_generator = train_datagen.flow_from_directory(
        'img_dataset/train', 
        target_size=(512, 512),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'img_dataset/validation',
        target_size=(512, 512),
        batch_size=batch_size,
        class_mode='categorical')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Found 27604 images belonging to 5 classes.
Found 5199 images belonging to 5 classes.


In [ ]:
# Let's start training!
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')

'''I trained the model on FloydHub, reaching an accuracy on the validation set of 0.9948'''

In [2]:
def load_trained_model(weights_path):
    final_model = create_model()
    final_model.load_weights(weights_path)
    return final_model

In [5]:
# You can find the trained model here in GitHub and test it!

load_model = load_trained_model("first_try.h5")
load_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [14]:
from random import shuffle

'''Loading and shuffling images from test directory with 4 as label'''
images = load_images_from_folder('img_dataset/test/4')
shuffle(images)

for i in range(10):
    img = images[i]
    if (img is not None):
        img = np.expand_dims(img, axis=0)
        result_class = load_model.predict_classes(img)
        print(result_class)

1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
1/1 [==============================] - 0s
[4]
